# What if my dataset isn't on the Hub?

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [ ]:
!pip install datasets evaluate transformers[sentencepiece]

In [ ]:
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-train.json.gz
!wget https://github.com/crux82/squad-it/raw/master/SQuAD_it-test.json.gz

`!` — означает, что команда выполняется в среде Jupyter Notebook или Google Colab как shell-команда.  
`wget` — утилита для скачивания файлов из интернета.  
`https: ...` — это URL-адрес файла, который будет загружен.
файл `SQuAD_it-train.json.gz` — сжатый JSON-файл (gzip), содержащий обучающую выборку итальянской версии датасета SQuAD (Stanford Question Answering Dataset).
(аналогично, для тестовых данных)

In [ ]:
!gzip -dkv SQuAD_it-*.json.gz

`!` — выполняет команду в Jupyter Notebook или Google Colab.  
`gzip` — утилита для сжатия и разжатия файлов в формате .gz.  
`-d (или --decompress)` — разархивирует (распаковывает) файл .gz, создавая несжатый файл.  
`-k (или --keep)` — сохраняет оригинальный сжатый файл (.gz) после распаковки.  
`-v (или --verbose)` — выводит информацию о процессе разархивации.  
`SQuAD_it-*.json.gz` — шаблон имени файла (использует * для подстановки), означающий "все файлы, начинающиеся с SQuAD_it- и заканчивающиеся на .json.gz".  
Что делает команда:  
Она распаковывает все файлы формата SQuAD_it-*.json.gz, оставляя как сжатую, так и несжатую версию (.json.gz и .json).

<font color="yellow">Чтобы загрузить файл JSON с помощью функции load_dataset(), нам просто нужно знать, имеем ли мы дело с обычным JSON (похожим на вложенный словарь) или строками JSON (JSON, разделенный строками). Как и многие наборы данных с ответами на вопросы, SQuAD-it использует вложенный формат, при этом весь текст хранится в поле данных. Это означает, что мы можем загрузить набор данных, указав аргумент поля следующим образом:

In [4]:
from datasets import load_dataset
squad_it_dataset = load_dataset("json", data_files="SQuAD_it-train.json", field="data")

Generating train split: 0 examples [00:00, ? examples/s]

По умолчанию загрузка локальных файлов создает объект DatasetDict с разделением поездов.

In [5]:
squad_it_dataset

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
})

In [ ]:
squad_it_dataset["train"][0]

Отлично, мы загрузили наш первый локальный набор данных! Но хотя это сработало для обучающего набора, на самом деле мы хотим включить и обучающее, и тестовое разделение в один объект DatasetDict, чтобы мы могли применить функции Dataset.map() к обоим разделениям одновременно. Для этого мы можем предоставить словарь аргументу data_files, который сопоставляет каждое имя разделения с файлом, связанным с этим разделением:

In [7]:
data_files = {"train": "SQuAD_it-train.json", "test": "SQuAD_it-test.json"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")
squad_it_dataset

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 442
    })
    test: Dataset({
        features: ['title', 'paragraphs'],
        num_rows: 48
    })
})

`data_files` — переменная, содержащая словарь (dict).  
Ключи словаря:  
`"train"` — ключ, обозначающий обучающий набор данных.  
`"test"` — ключ, обозначающий тестовый набор данных.  
Значения:  
`"SQuAD_it-train.json"` — путь к файлу с обучающим датасетом.  
`"SQuAD_it-test.json"` — путь к файлу с тестовым датасетом.

<font color='yellow'>Скрипты загрузки в 🤗 Datasets на самом деле поддерживают автоматическую распаковку входных файлов, поэтому мы могли бы пропустить использование gzip, указав аргумент data_files непосредственно на сжатые файлы:

In [ ]:
data_files = {"train": "SQuAD_it-train.json.gz", "test": "SQuAD_it-test.json.gz"}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")

Это может быть полезно, если вы не хотите вручную распаковывать много файлов GZIP. Автоматическая распаковка также применяется к другим распространенным форматам, таким как ZIP и TAR, поэтому вам просто нужно указать data_files на сжатые файлы, и все готово!

## Загрузка удаленного набора данных

загрузка удаленных файлов:вместо указания пути к локальным файлам мы указываем аргумент data_files load_dataset() на один или несколько URL-адресов, где хранятся удаленные файлы.

In [ ]:
url = "https://github.com/crux82/squad-it/raw/master/"
data_files = {
    "train": url + "SQuAD_it-train.json.gz",
    "test": url + "SQuAD_it-test.json.gz",
}
squad_it_dataset = load_dataset("json", data_files=data_files, field="data")